In [1]:
import matplotlib.pyplot as plt 

import numpy as np 
from sklearn.datasets import fetch_openml
import random 

import cntk.tests.test_utils
from sklearn.preprocessing import OneHotEncoder

import cntk as C

c:\Users\Natto\anaconda3\envs\cnntk\lib\site-packages\cntk\cntk_py_init.py:84: UserWarning: 

################################################ Missing optional dependency (GPU-Specific) ################################################
   CNTK may crash if the component that depends on those dependencies is loaded.
   Visit https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages for more information.
############################################################################################################################################
If you intend to use CNTK without GPU support, you can ignore the (likely) GPU-specific warning!
############################################################################################################################################

  warnings.warn(WARNING_MSG_GPU_ONLY % ('GPU-Specific', 'https://docs.microsoft.com/en-us/cognitive-toolkit/Setup-Windows-Python#optional-gpu-specific-packages'))


In [35]:
num_samples = 60000
batch_size = 64 
learning_rate = 0.1 

class Batch_Reader(object):
    def __init__(self, data, label):
        self.data = data 
        self.label = label
        self.num_sample = data.shape[0]

    def next_batch(self, batch_size):
        index = random.sample(range(self.num_sample), batch_size)
        return self.data[index,:].astype(float),self.label[index,:].astype(float)



In [3]:
mnist = fetch_openml('mnist_784')

In [4]:
print(mnist)

{'data':        pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4         0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69996     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69997     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69998     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
69999     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

       pixel10  ..

In [5]:
train_data = mnist.data[:num_samples:]
train_label = mnist.target[:num_samples]
test_data = mnist.data[num_samples::]
test_label = mnist.target[num_samples:]

In [6]:
print(len(test_label))

10000


In [7]:
train_label_array = np.array(train_label)
enc = OneHotEncoder()
enc.fit(train_label_array[:,None])
train_encoded = enc.transform(train_label_array[:,None]).toarray()

In [8]:
print(train_encoded)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [9]:
train_reader = Batch_Reader(train_data, train_encoded)

In [10]:
test_label_array = np.array(test_label)
enc = OneHotEncoder()
enc.fit(test_label_array[:,None])
test_encoded = enc.transform(test_label_array[:,None]).toarray()

test_reader = Batch_Reader(test_data, test_encoded)

In [11]:
dimensions = 784
classes = 10
hidden_layers = 3
hidden_layers_neurons = 400

In [12]:
input = C.input_variable(dimensions)
label = C.input_variable(classes)

In [13]:
def create_model(features):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.ops.relu):
        hidden_out = features
        for _ in range(hidden_layers):
            hidden_out = C.layers.Dense(hidden_layers_neurons)(hidden_out)
            network_output = C.layers.Dense(classes, activation= None)(hidden_out)

            return network_output


In [14]:
model = create_model(input/255.0)

In [15]:
print(model)

Composite(Tensor[784]) -> Tensor[10]


# Training the CNTK Model 

In [16]:
loss = C.cross_entropy_with_softmax(model, label)
label_error = C.classification_error(model, label)

In [17]:
lrs = C.learning_rate_schedule(learning_rate, C.UnitType.minibatch)
learner = C.sgd(model.parameters, lrs)
trainer = C.Trainer(model, (loss, label_error), [learner])

In [34]:
epochs = 10
num_iters = (num_samples * epochs) / batch_size

for i in range(int(num_iters)):
    batch_data, batch_label = train_reader.next_batch(batch_size=batch_size)
    arguments = {input: batch_data, label: batch_label}
    trainer.train_minibatch(arguments=arguments)

    if i % 1000 == 0:
        training_loss = False
        evaluation_error = False
        training_loss = trainer.previous_minibatch_loss_average
        evaluation_error = trainer.previous_minibatch_evaluation_average
        print('{0}: ,Loss: {1:.3f}, Error: {2:.2f}%'.format(i, training_loss, evaluation_error * 100))

TypeError: '([1850, 58080, 23408, 26209, 17874, 39016, 30380, 21412, 1294, 35096, 22816, 44182, 55578, 29462, 28081, 616, 10053, 13273, 58672, 5110, 48932, 20906, 22183, 17675, 8750, 50581, 56272, 50471, 21095, 53013, 13589, 24167, 2794, 23559, 46575, 2627, 19074, 26746, 19846, 57960, 19900, 50458, 56287, 21233, 26898, 13180, 32279, 15329, 17184, 10554, 44219, 59471, 13216, 39928, 58348, 10710, 23719, 5470, 26442, 51555, 40329, 294, 44207, 58372], slice(None, None, None))' is an invalid key

AttributeError: 'Batch_Reader' object has no attribute 'type'